# Model 8: Predict Mass Class (10) Based on Visual Morphology (clean galaxy image)


In [2]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import ImageFolder 
from torchvision.io import read_image
from torchvision.io import decode_image
from torchvision.models import vgg19
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image as image_PIL
import PIL 
from PIL import ImageShow
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, EigenGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

#set config attributes and turn on global downloads of Marvin data
config.setRelease('DR15')
config.mode = 'local'
config.download = True



%matplotlib qt



[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Importing Data from Schema Table

In [3]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1


#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

galaxy_list=np.unique(galaxy_list)



galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 



galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

galaxies=galaxies.sort_values(by=['plateifu']) #Sorting galaxies by plateifu to match ImageFolder Output 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

mangaid=galaxies.loc[:,'mangaid']

plateifu=galaxies.loc[:,'plateifu']

if plateifu.all()==np.unique(plateifu).all() and mangaid.all()==np.unique(mangaid.all()):
    print('Is Unique List of Galaxies')
    
else:
    print('Try Again')


z=galaxies.loc[:,'z']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)

bars=np.linspace(np.min(log_mass),11.25,num=10)
plt.hist(log_mass)
plt.vlines(bars,ymin=0,ymax=800)


mass_cat = np.zeros_like(log_mass)
mass_cat[(log_mass> bars[0]) & (log_mass<bars[1])]=int(0)
mass_cat[(log_mass> bars[1]) & (log_mass<bars[2])]=int(1)
mass_cat[(log_mass> bars[2]) & (log_mass<bars[3])]=int(2)
mass_cat[(log_mass> bars[3]) & (log_mass<bars[4])]=int(3)
mass_cat[(log_mass> bars[4]) & (log_mass<bars[5])]=int(4)
mass_cat[(log_mass> bars[5]) & (log_mass<bars[6])]=int(5)
mass_cat[(log_mass> bars[6]) & (log_mass<bars[7])]=int(6)
mass_cat[(log_mass> bars[7]) & (log_mass<bars[8])]=int(7)
mass_cat[(log_mass> bars[8]) & (log_mass<bars[9])]=int(8)
mass_cat[log_mass> bars[9]] = int(9)


print(np.mean(mass_cat))


Is Unique List of Galaxies
4.226525494566732


In [ ]:
print(plateifu.iloc[987])

In [ ]:
for i in range(len(galaxies)):
    im = Image(plateifu.iloc[i])


    # generate a new image
    # inputs are height and width in arcsec, and a arcsec/pixel scale
    im.get_new_cutout(50, 50, scale=0.089)

    # plot the new image cutout
    im.plot()
    plt.savefig('/home/juanp/Documents/Clean Images (Marvin Size)/Images/'+str(i))
    plt.close()


# Importing Images from their Downloaded Locations 


In [ ]:
# image_locations=[]
# for i in range (len(galaxy_list)):
#     image_locations.append(Image(galaxy_list[i]).filename)
    
# image_locations=np.array(image_locations,dtype=str)
# np.savetxt('Image Directories',image_locations,fmt='%s')

image_locations=np.loadtxt('Image Directories',dtype=str)



In [1]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    from PIL import Image, ImageOps 
    
    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)
    
    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)
    
    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
  
    # return the resized image
    return new_image



# Putting the Images into DataLoaders

In [4]:
# img_size=(128,128)

# image=ImageFolder('/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/') #Picks up 3590 pictures, directory list has lenght 3637 however 

# torch.manual_seed(15)
# np.random.seed(15)

image_directory='/home/juanp/Documents/Clean Images (Marvin Size)'
classes= 10



batch_size=50
image_copies=11

def load_dataset(data_path):
    # Load all the images
    transformation = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        #Rotates the image by some angle
        transforms.RandomRotation(0,360),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        # transform=transformation
    )
    
    #This loop transforms the images and assigns them the correct label 

    full_dataset_v2 = []   
    for i in range(len(full_dataset)): 
        for j in range(image_copies):
            temp=transformation(resize_image(full_dataset[i][0]))
            full_dataset_v2.append((temp,mass_cat[i])) 
    
    full_dataset=full_dataset_v2

    print(len(full_dataset))


    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    
    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader




#####################################################################################################



# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(image_directory)
batch_size = train_loader.batch_size
print("Data loaders ready to read", image_directory)



39479
Data loaders ready to read /home/juanp/Documents/Clean Images (Marvin Size)


# Defining the Model 

In [5]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        
        # In the init function, we define each layer we will use in our model
        
        # Our images are RGB, so we have input channels = 3. 
        # We will apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 32 x 32, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=24*32*32, out_features=num_classes)

#         self.fc1= nn.Linear(in_features=10, out_features=1)


    def forward(self, x):
        
        
        # In the forward function, pass the data through the layers we defined in the init function
        # print(x)
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x=self.conv1(x)
        
        x=self.pool(x)
       

        x=F.relu(x)

        
      
 
     
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x))) 
        
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
        # print(x.shape)
        x = x.view(-1, 24*32*32)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        
        # Return class probabilities via a log_softmax function
#         x=F.relu(x) 
        
#         x= self.fc1(x)
        # print(x)
        return torch.log_softmax(x, dim=1)
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
classes=10
model = Net(num_classes=classes).to('cuda')

print(model)

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=24576, out_features=10, bias=True)
)


# Creating Training Loop/Function

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):

        target=target.long()
        # print(target.shape)
#         target=target.reshape(-1,1)
        
        # Use the CPU or GPU as appropriate
        # Recall that GPU is optimized for the operations we are dealing with
        data, target = data.to(device), target.to(device)
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
    
        output = model(data)

        
        # Get the loss
        loss = loss_criteria(output, target)
#         if batch_idx==0:
#             print(output,target,output-target,loss)
#         # Keep a running total
        train_loss += loss.item()
        
        # Backpropagate
        loss.backward()
        optimizer.step()

       
        
        # Print metrics so we see some progress
        # print('\tTraining batch {} Loss: {:.6f}'.format(batch_idx + 1, loss.item()))
            
    # return average loss for the epoch
    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

# Create Test Function

In [7]:
def test(model, device, test_loader):
    # Switch the model to evaluation mode (so we don't backpropagate or drop)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        batch_count = 0
        for data, target in test_loader:
            target=target.long()
#             target=target.reshape(-1,1)
            
            batch_count += 1
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += loss_criteria(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += torch.sum(target==predicted).item()

    # Calculate the average loss and total accuracy for this epoch
    avg_loss = test_loss / batch_count
    print('Validation set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # return average loss for the epoch
    return avg_loss

# Training the Model

In [8]:
# Use an "Adam" optimizer to adjust weights
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the loss criteria
loss_criteria = nn.CrossEntropyLoss()

# Track metrics in these arrays
epoch_nums = []
training_loss = []
validation_loss = []




# Train over 10 epochs (We restrict to 10 for time issues)
epochs = 1000
print('Training on', device)
for epoch in range(1, epochs + 1):
        print(epoch)
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)
torch.save(model.state_dict(),'/home/juanp/Documents/SURP-2021/Models/Model 8 (Clean Images and Mass)/MarvinSize(11 Copies at 128x128)')


Training on cuda
1
Epoch: 1
Training set: Average loss: 2.225734
Validation set: Average loss: 2.181924, Accuracy: 2186/11844 (18%)

2
Epoch: 2
Training set: Average loss: 2.100320
Validation set: Average loss: 2.066936, Accuracy: 2868/11844 (24%)

3
Epoch: 3
Training set: Average loss: 1.960704
Validation set: Average loss: 1.980918, Accuracy: 3477/11844 (29%)

4
Epoch: 4
Training set: Average loss: 1.845545
Validation set: Average loss: 1.900942, Accuracy: 4002/11844 (34%)

5
Epoch: 5
Training set: Average loss: 1.746200
Validation set: Average loss: 1.816937, Accuracy: 4459/11844 (38%)

6
Epoch: 6
Training set: Average loss: 1.649770
Validation set: Average loss: 1.736744, Accuracy: 5028/11844 (42%)

7
Epoch: 7
Training set: Average loss: 1.553989
Validation set: Average loss: 1.639801, Accuracy: 5650/11844 (48%)

8
Epoch: 8
Training set: Average loss: 1.456084
Validation set: Average loss: 1.553446, Accuracy: 6183/11844 (52%)

9
Epoch: 9
Training set: Average loss: 1.376006
Validat

Validation set: Average loss: 0.821542, Accuracy: 10268/11844 (87%)

71
Epoch: 71
Training set: Average loss: 0.513665
Validation set: Average loss: 0.839029, Accuracy: 10267/11844 (87%)

72
Epoch: 72
Training set: Average loss: 0.511794
Validation set: Average loss: 0.835015, Accuracy: 10278/11844 (87%)

73
Epoch: 73
Training set: Average loss: 0.521519
Validation set: Average loss: 0.836791, Accuracy: 10267/11844 (87%)

74
Epoch: 74
Training set: Average loss: 0.511723
Validation set: Average loss: 0.834576, Accuracy: 10300/11844 (87%)

75
Epoch: 75
Training set: Average loss: 0.515513
Validation set: Average loss: 0.841402, Accuracy: 10301/11844 (87%)

76
Epoch: 76
Training set: Average loss: 0.498294
Validation set: Average loss: 0.827841, Accuracy: 10292/11844 (87%)

77
Epoch: 77
Training set: Average loss: 0.500911
Validation set: Average loss: 0.821298, Accuracy: 10312/11844 (87%)

78
Epoch: 78
Training set: Average loss: 0.500668
Validation set: Average loss: 0.833091, Accuracy

Training set: Average loss: 0.416654
Validation set: Average loss: 0.823561, Accuracy: 10445/11844 (88%)

140
Epoch: 140
Training set: Average loss: 0.420553
Validation set: Average loss: 0.835419, Accuracy: 10442/11844 (88%)

141
Epoch: 141
Training set: Average loss: 0.415124
Validation set: Average loss: 0.856673, Accuracy: 10448/11844 (88%)

142
Epoch: 142
Training set: Average loss: 0.409166
Validation set: Average loss: 0.856999, Accuracy: 10449/11844 (88%)

143
Epoch: 143
Training set: Average loss: 0.407865
Validation set: Average loss: 0.856339, Accuracy: 10436/11844 (88%)

144
Epoch: 144
Training set: Average loss: 0.413997
Validation set: Average loss: 0.818921, Accuracy: 10457/11844 (88%)

145
Epoch: 145
Training set: Average loss: 0.409357
Validation set: Average loss: 0.838549, Accuracy: 10449/11844 (88%)

146
Epoch: 146
Training set: Average loss: 0.404986
Validation set: Average loss: 0.832804, Accuracy: 10431/11844 (88%)

147
Epoch: 147
Training set: Average loss: 0.41

Training set: Average loss: 0.358536
Validation set: Average loss: 0.854174, Accuracy: 10449/11844 (88%)

208
Epoch: 208
Training set: Average loss: 0.368213
Validation set: Average loss: 0.859143, Accuracy: 10459/11844 (88%)

209
Epoch: 209
Training set: Average loss: 0.361106
Validation set: Average loss: 0.906128, Accuracy: 10448/11844 (88%)

210
Epoch: 210
Training set: Average loss: 0.363607
Validation set: Average loss: 0.853814, Accuracy: 10453/11844 (88%)

211
Epoch: 211
Training set: Average loss: 0.365881
Validation set: Average loss: 0.851344, Accuracy: 10457/11844 (88%)

212
Epoch: 212
Training set: Average loss: 0.362862
Validation set: Average loss: 0.865257, Accuracy: 10471/11844 (88%)

213
Epoch: 213
Training set: Average loss: 0.363318
Validation set: Average loss: 0.890818, Accuracy: 10471/11844 (88%)

214
Epoch: 214
Training set: Average loss: 0.371250
Validation set: Average loss: 0.870813, Accuracy: 10459/11844 (88%)

215
Epoch: 215
Training set: Average loss: 0.36

Training set: Average loss: 0.344394
Validation set: Average loss: 0.892152, Accuracy: 10469/11844 (88%)

276
Epoch: 276
Training set: Average loss: 0.338598
Validation set: Average loss: 0.875031, Accuracy: 10460/11844 (88%)

277
Epoch: 277
Training set: Average loss: 0.334381
Validation set: Average loss: 0.881146, Accuracy: 10477/11844 (88%)

278
Epoch: 278
Training set: Average loss: 0.351682
Validation set: Average loss: 0.889390, Accuracy: 10465/11844 (88%)

279
Epoch: 279
Training set: Average loss: 0.342583
Validation set: Average loss: 0.875794, Accuracy: 10465/11844 (88%)

280
Epoch: 280
Training set: Average loss: 0.335962
Validation set: Average loss: 0.893863, Accuracy: 10473/11844 (88%)

281
Epoch: 281
Training set: Average loss: 0.335069
Validation set: Average loss: 0.853171, Accuracy: 10472/11844 (88%)

282
Epoch: 282
Training set: Average loss: 0.333349
Validation set: Average loss: 0.887704, Accuracy: 10480/11844 (88%)

283
Epoch: 283
Training set: Average loss: 0.33

...

Training set: Average loss: 0.279533
Validation set: Average loss: 0.970456, Accuracy: 10474/11844 (88%)

684
Epoch: 684
Training set: Average loss: 0.272927
Validation set: Average loss: 1.016764, Accuracy: 10481/11844 (88%)

685
Epoch: 685
Training set: Average loss: 0.280255
Validation set: Average loss: 0.942980, Accuracy: 10481/11844 (88%)

686
Epoch: 686
Training set: Average loss: 0.269770
Validation set: Average loss: 0.976072, Accuracy: 10484/11844 (89%)

687
Epoch: 687
Training set: Average loss: 0.272267
Validation set: Average loss: 1.026894, Accuracy: 10490/11844 (89%)

688
Epoch: 688
Training set: Average loss: 0.283351
Validation set: Average loss: 0.955339, Accuracy: 10475/11844 (88%)

689
Epoch: 689
Training set: Average loss: 0.272500
Validation set: Average loss: 0.968173, Accuracy: 10476/11844 (88%)

690
Epoch: 690
Training set: Average loss: 0.267783
Validation set: Average loss: 0.910829, Accuracy: 10461/11844 (88%)

691
Epoch: 691
Training set: Average loss: 0.27

Training set: Average loss: 0.274404
Validation set: Average loss: 1.024607, Accuracy: 10486/11844 (89%)

752
Epoch: 752
Training set: Average loss: 0.267609
Validation set: Average loss: 0.980749, Accuracy: 10499/11844 (89%)

753
Epoch: 753
Training set: Average loss: 0.265299
Validation set: Average loss: 0.989334, Accuracy: 10486/11844 (89%)

754
Epoch: 754
Training set: Average loss: 0.273064
Validation set: Average loss: 1.018450, Accuracy: 10489/11844 (89%)

755
Epoch: 755
Training set: Average loss: 0.262696
Validation set: Average loss: 0.978220, Accuracy: 10470/11844 (88%)

756
Epoch: 756
Training set: Average loss: 0.274801
Validation set: Average loss: 0.932603, Accuracy: 10489/11844 (89%)

757
Epoch: 757
Training set: Average loss: 0.268960
Validation set: Average loss: 0.983120, Accuracy: 10486/11844 (89%)

758
Epoch: 758
Training set: Average loss: 0.272366
Validation set: Average loss: 0.994447, Accuracy: 10481/11844 (88%)

759
Epoch: 759
Training set: Average loss: 0.27

Training set: Average loss: 0.273008
Validation set: Average loss: 1.015572, Accuracy: 10496/11844 (89%)

820
Epoch: 820
Training set: Average loss: 0.269427
Validation set: Average loss: 0.962672, Accuracy: 10493/11844 (89%)

821
Epoch: 821
Training set: Average loss: 0.264873
Validation set: Average loss: 0.950624, Accuracy: 10487/11844 (89%)

822
Epoch: 822
Training set: Average loss: 0.264052
Validation set: Average loss: 0.964142, Accuracy: 10479/11844 (88%)

823
Epoch: 823
Training set: Average loss: 0.271301
Validation set: Average loss: 0.954725, Accuracy: 10497/11844 (89%)

824
Epoch: 824
Training set: Average loss: 0.275639
Validation set: Average loss: 0.947914, Accuracy: 10489/11844 (89%)

825
Epoch: 825
Training set: Average loss: 0.270773
Validation set: Average loss: 0.996658, Accuracy: 10472/11844 (88%)

826
Epoch: 826
Training set: Average loss: 0.269592
Validation set: Average loss: 0.991997, Accuracy: 10489/11844 (89%)

827
Epoch: 827
Training set: Average loss: 0.26

Training set: Average loss: 0.268205
Validation set: Average loss: 1.002962, Accuracy: 10491/11844 (89%)

888
Epoch: 888
Training set: Average loss: 0.262310
Validation set: Average loss: 1.005748, Accuracy: 10484/11844 (89%)

889
Epoch: 889
Training set: Average loss: 0.260957
Validation set: Average loss: 1.005998, Accuracy: 10466/11844 (88%)

890
Epoch: 890
Training set: Average loss: 0.263953
Validation set: Average loss: 0.995325, Accuracy: 10495/11844 (89%)

891
Epoch: 891
Training set: Average loss: 0.260369
Validation set: Average loss: 0.958609, Accuracy: 10488/11844 (89%)

892
Epoch: 892
Training set: Average loss: 0.271381
Validation set: Average loss: 0.983036, Accuracy: 10476/11844 (88%)

893
Epoch: 893
Training set: Average loss: 0.263507
Validation set: Average loss: 0.996226, Accuracy: 10490/11844 (89%)

894
Epoch: 894
Training set: Average loss: 0.261939
Validation set: Average loss: 1.013744, Accuracy: 10486/11844 (89%)

895
Epoch: 895
Training set: Average loss: 0.26

Training set: Average loss: 0.250605
Validation set: Average loss: 0.994685, Accuracy: 10479/11844 (88%)

956
Epoch: 956
Training set: Average loss: 0.257624
Validation set: Average loss: 0.976362, Accuracy: 10506/11844 (89%)

957
Epoch: 957
Training set: Average loss: 0.252028
Validation set: Average loss: 0.952188, Accuracy: 10483/11844 (89%)

958
Epoch: 958
Training set: Average loss: 0.264844
Validation set: Average loss: 1.001392, Accuracy: 10480/11844 (88%)

959
Epoch: 959
Training set: Average loss: 0.259387
Validation set: Average loss: 1.064702, Accuracy: 10482/11844 (89%)

960
Epoch: 960
Training set: Average loss: 0.261490
Validation set: Average loss: 0.949208, Accuracy: 10492/11844 (89%)

961
Epoch: 961
Training set: Average loss: 0.257607
Validation set: Average loss: 1.003540, Accuracy: 10482/11844 (89%)

962
Epoch: 962
Training set: Average loss: 0.257370
Validation set: Average loss: 1.061811, Accuracy: 10487/11844 (89%)

963
Epoch: 963
Training set: Average loss: 0.26

In [9]:
plt.figure(figsize=(15,15))
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

In [10]:


# Defining Labels and Predictions
import seaborn as sns
classes=[0,1,2,3,4,5,6,7,8,9]
truelabels = []
predictions = []
model.eval()
print("Getting predictions from test set...")
for data, target in test_loader:
    data, target= data.to('cuda'), target.to('cuda')
    for label in target.data.cpu().numpy():
        truelabels.append(label)
    for prediction in model(data).data.cpu().numpy().argmax(1):
        predictions.append(prediction) 

# Plot the confusion matrix
cm = confusion_matrix(truelabels, predictions)
tick_marks = np.arange(len(classes))

df_cm = pd.DataFrame(cm, index = classes, columns = classes)
plt.figure(figsize = (7,7))
sns.heatmap(df_cm, annot=True, cmap=plt.cm.Blues, fmt='g')
plt.xlabel("Predicted Shape", fontsize = 20)
plt.ylabel("True Shape", fontsize = 20)
plt.show()



Getting predictions from test set...


# Loading up Saved Model

In [11]:
model.load_state_dict(torch.load('/home/juanp/Documents/SURP-2021/Models/Model 8 (Clean Images and Mass)/MarvinSize(11 Copies at 128x128)'))
model.eval()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=24576, out_features=10, bias=True)
)

In [ ]:
from torchvision.models import resnet50
model50=resnet50 
print(model50)

In [45]:
image=ImageFolder('/home/juanp/Documents/Clean Images (Marvin Size)')
plt.imshow(image[46][0])
# print(np.shape(image[14][0]))
# PIL.ImageShow.show(resize_image(image[14][0]))

In [ ]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

trans=transforms.ToTensor()

model = model
target_layer = model.conv2

input_tensor =trans(resize_image(image[14][0]))

print(np.max(input_tensor.detach().numpy()))


input_tensor2=np.transpose(input_tensor,(1,2,0))
print(input_tensor2.shape)

input_tensor=input_tensor.unsqueeze(0)
input_tensor=input_tensor.to('cuda')

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layer=target_layer, use_cuda='args.use_cuda')

# If target_category is None, the highest scoring category
# will be used for every image in the batch.
# target_category can also be an integer, or a list of different integers
# for every image in the batch.
target_category = None

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0,:]
visualization = show_cam_on_image(input_tensor2.detach().numpy(), grayscale_cam)
plt.subplot(1,2,1)
plt.imshow(visualization)
plt.subplot(1,2,2)
plt.imshow(resize_image(image[14][0]))

In [47]:


def galaxy_grad_cam_resize(input_image,file_name,model=model):
  
    trans=transforms.ToTensor()

    target_layer = model.conv2

    input_tensor =trans(resize_image(input_image))

    print(np.max(input_tensor.detach().numpy()))


    input_tensor2=np.transpose(input_tensor,(1,2,0))
    print(input_tensor2.shape)

    input_tensor=input_tensor.unsqueeze(0)
    input_tensor=input_tensor.to('cuda')

    # Construct the CAM object once, and then re-use it on many images:
    torch.autograd.set_detect_anomaly(True)
    cam = EigenCAM(model=model, target_layer=target_layer, use_cuda='args.use_cuda')
    

    # If target_category is None, the highest scoring category
    # will be used for every image in the batch.
    # target_category can also be an integer, or a list of different integers
    # for every image in the batch.
    target_category = None

    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0,:]
    visualization = show_cam_on_image(input_tensor2.detach().numpy(), grayscale_cam)
    plt.figure(figsize=(18,9))
    plt.subplot(1,2,1)
    plt.imshow(visualization)
    plt.subplot(1,2,2)
    plt.imshow(resize_image(input_image))
    plt.savefig('/home/juanp/Documents/SURP-2021/Plots/Grad-Cam/Mass (Marvin Size 11 Copies)/'+file_name+'.png')
    # plt.close()

i=17
galaxy_grad_cam_resize(image[i][0],'Image_Eigen '+str(i))

1.0
torch.Size([128, 128, 3])


In [37]:
print(plateifu.iloc[4])

10001-1901


In [ ]:


def galaxy_grad_cam(input_image,file_name,model=model):
  
    trans=transforms.ToTensor()

    target_layer = model.conv2

    input_tensor =trans(input_image)

    print(np.max(input_tensor.detach().numpy()))


    input_tensor2=np.transpose(input_tensor,(1,2,0))
    print(input_tensor2.shape)

    input_tensor=input_tensor.unsqueeze(0)
    input_tensor=input_tensor.to('cuda')

    # Construct the CAM object once, and then re-use it on many images:
    torch.autograd.set_detect_anomaly(True)
    cam = EigenCAM(model=model, target_layer=target_layer, use_cuda='args.use_cuda')
    

    # If target_category is None, the highest scoring category
    # will be used for every image in the batch.
    # target_category can also be an integer, or a list of different integers
    # for every image in the batch.
    target_category = None

    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0,:]
    visualization = show_cam_on_image(input_tensor2.detach().numpy(), grayscale_cam)
    plt.figure(figsize=(18,9))
    plt.subplot(1,2,1)
    plt.imshow(visualization)
    plt.subplot(1,2,2)
    plt.imshow(input_image)
    plt.savefig('/home/juanp/Documents/SURP-2021/Plots/Grad-Cam/Mass (Marvin Size)/'+file_name+'.png')
    # plt.close()

i=1854
galaxy_grad_cam(image[i][0],'Image_Eigen'+str(i))

1.0
torch.Size([600, 600, 3])


In [ ]:
print(input_tensor.cpu().detach().numpy().shape)
print(grayscale_cam.shape)
print(input_tensor2.detach().numpy().shape)
print(visualization.shape)